# **HDT1 - Modelación y simulación**

### integrantes: 
- Francis Aguilar - 22243
- Diego García - 22404
- Angela García -22869

# Ejercicio 1


1. **Defina los stocks (reservas) y los flujos en la dinámica de sistemas. Proporcione un ejemplo real (p. ej., modelado de pandemias).**

2. **Explique la diferencia entre ciclos de retroalimentación de equilibrio (negativos) y de refuerzo (positivos). Dé un ejemplo de cada uno en el contexto de la COVID-19.**

3. **¿Por qué las ecuaciones diferenciales son fundamentales en los modelos de dinámica de sistemas? ¿Cómo capturan la no linealidad?** 

4. **Dibuje un diagrama de bucle causal para la ocupación de camas hospitalarias durante una pandemia. Etiquete las reservas (p. ej., "Pacientes de UCI"), los flujos (p. ej., "Admisiones") y los bucles de retroalimentación (p. ej., "Hacinamiento → Estancias más prolongadas → Disponibilidad reducida de camas").**

# Ejercicio 2


1. **¿Qué es un agente en el modelo ABM? ¿En qué se diferencia de un "stock" en la dinámica de sistemas?**

    Un agente en el modelo ABM es una entidad física o virtual que toman decisiones de manera autónoma, estos pueden ser representados como átomos, células, animales, gente e organizaciones. Estos dependen de la aplicación que se les de, además tienen recursos propios, objetivos, capacidades sensoriales.  
    Lo que se diferencia de un stock en la dinámica de sistemas es que un stock es una variable acumulatica, que se va llenando o vaciando, en cambio de un agente ABM que tiene comportamientos y decisiones, no es que acumule cosas por si mismos.
    Por otra parte los stocks, no interactuan y no deciden nada, estos representan cantidades que cambian con el tiempo. A diferencia de un agente que tiene objetivos, estrategia y memoria. 


2. **Describa cómo se representa la heterogeneidad (p. ej., edad, ingresos) en el ABM en comparación con la dinámica de sistemas.** 
    La heterogeneidad, en los ABM es explicitá y granular, porque cada agente puede tener atributos únicos. Cada agente tiene propiedades individuales y estas pueden llegar a influir en su comportamiento, decisiones y evolución. Además, se pueden usar distribuciones estadísticas para asignar valores iniciales. Y por último, pueden cambiar con el tiempo, ya que ganan experiencia o cambian de estrategia. 
    Y en la dinámica de sistemas la heterogeneidad se representa de forma agregada o segmentada, no es individualmente. Se crean stocks separados para cada grupo y cada stock tienen flujos que representan transiciones entre grupos. La diferencia de grupos se modelan con parámetros distintos.


3. **¿Por qué el ABM suele ser estocástico? ¿Cómo afecta esto a la interpretación del modelo?**
    El ABM suele ser estocástico ya que es complejo y muestra la incertidumbre y la variavilidad. Este tiene variabilidad individual, ambientes inciertos, interacciones impredecibles y exploración de escenarios. Al tener todas estas caracteristicas, dan resultados no deterministas, ya que una misma configuración inicial puede producir resultados diferentes en cada ejecución. Otra forma en la que afecta, es en el análisis estadístico, en lugar de una única predicción se analizan tendencias, medias, desviaciones estandar y otras estadisticas.  

4. **Imagine modelar una campaña de vacunación. Compare cómo la dinámica de sistemas (compartimentos agregados) y el análisis de mercado (agentes individuales) representarían la reticencia a la vacunación. ¿Qué enfoque captura mejor las redes sociales?**

    El enfoque que capruta mejor las redes sociales puede ser el de ABM dya que este al tener un nivel de análisis individual la reticencia se representa con un agente que es unas persona con atributos únicos, como la edad, sexo, educación y otros. Este también puede depender de experiencias previas, creencias personales o la influencia de otros agentes y las decisiones de vacunacion pueden ser moldeadas como probabilidades. Además, las redes sociales son una red de interacción ya que se comparte información.



# Ejercicio 3

In [ ]:
#1. dinamica del sistema, nivel agregado - modelo SIR

#b. ecuaciones diferenciales para modelar las transiciones agregadas entre comportamientos
#c. considerar los parámetros, como tasa de transmisión, tasa de recuperación y afectan a la curva epidemica

class SIR:
    def __init__(self, poblacion, infectados_iniciales, recuperados_iniciales, beta, gamma):
        self.N = poblacion
        self.beta = beta
        self.gamma = gamma

        self.S = poblacion - infectados_iniciales - recuperados_iniciales
        self.I = infectados_iniciales
        self.R = recuperados_iniciales

        #historial
        self.historial = {
            'S' : [self.S],
            'I': [self.I],
            'R': [self.R]
        }


    def paso(self, dt=1):
        nuevos_infectados = self.beta * self.S * self.I / self.N
        nuevos_recuperados= self.gamma*self.I

        #actualizacion
        dS = -(nuevos_infectados * dt)
        dI= (nuevos_infectados-nuevos_recuperados) * dt
        dR= nuevos_recuperados * dt

        self.S += dS
        self.I += dI
        self.R += dR

        #guardarlos
        self.historial['S'].append(self.S)
        self.historial['I'].append(self.I)
        self.historial['R'].append(self.R)

    def simular(self, dias, dt=1):
        pasos= int(dias/dt)
        for x in range(pasos):
            self.paso(dt)

    def get_historial(self):
        return self.historial




In [ ]:
#modelo SIR
sir = SIR(poblacion=1000, infectados_iniciales=1, recuperados_iniciales=0, beta=0.3, gamma=0.1)
sir.simular(dias=160)

In [3]:
## MODELO BASE DE AGENTE

import random

class Agente:
    def __init__(self, estado='S'):
        self.estado = estado
        self.dias_infectado = 0

        
class ModeloABM:
    def __init__(self, poblacion, infectados_iniciales, beta, dias_recuperacion):
        self.poblacion = [Agente() for _ in range(poblacion)]
        self.beta = beta  # probabilidad de contagio por contacto
        self.dias_recuperacion = dias_recuperacion

        # infectar a algunos agentes al inicio
        infectados = random.sample(self.poblacion, infectados_iniciales)
        for agente in infectados:
            agente.estado = 'I'

        self.historial = {
            'S': [],
            'I': [],
            'R': []
        }

    def contar_estados(self):
        s = sum(1 for a in self.poblacion if a.estado == 'S')
        i = sum(1 for a in self.poblacion if a.estado == 'I')
        r = sum(1 for a in self.poblacion if a.estado == 'R')
        self.historial['S'].append(s)
        self.historial['I'].append(i)
        self.historial['R'].append(r)

    def paso(self):
        # Simular interacciones entre pares de agentes
        for agente in self.poblacion:
            if agente.estado == 'S':
                otros = random.sample(self.poblacion, 5)  # 5 interacciones por día
                for otro in otros:
                    if otro.estado == 'I':
                        if random.random() < self.beta:
                            agente.estado = 'I'
                            break

        # Actualizar días infectado y recuperación
        for agente in self.poblacion:
            if agente.estado == 'I':
                agente.dias_infectado += 1
                if agente.dias_infectado >= self.dias_recuperacion:
                    agente.estado = 'R'

        self.contar_estados()

    def simular(self, dias):
        for _ in range(dias):
            self.paso()

    def get_historial(self):
        return self.historial
